In [1]:
# filtering future warnings/ ignoring future warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# importing keras for CNN and similar required libraries
import keras
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from keras.models import Model, Sequential
from keras import optimizers
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os, random
from PIL import Image


Using TensorFlow backend.


In [2]:
#configuring parameters here itself:
IMAGE_SIZE = [30, 30, 3]

epochs = 30
batch_size = 32
#path to image files directory
train_path = '/home/sree/Music/cnntrain'
valid_path = '/home/sree/Music/cnnvalidation'
test_path = '/home/sree/Music/test'
# used for getting number of files
train_files = glob(train_path + '/*/*.jp*g')
valid_files = glob(valid_path + '/*/*.jp*g')

# used for getting number of classes
classes = glob(train_path + '/*')
print('no of classes used for training :',len(classes))

no of classes used for training : 10


In [3]:
# developing CNN layers
model = Sequential()
#1st convolution set of 2 conv layers followed by max pooling has 32 filters each
model.add(Conv2D(32, (3, 3), activation='relu', name='set1_conv1', input_shape=IMAGE_SIZE, padding= 'same'))
model.add(Conv2D(32, (3, 3), activation='relu',name='set1_conv2'))
model.add(MaxPooling2D(pool_size=(2, 2), name='set1_maxpool1'))
model.add(Dropout(0.25))

#2nd convolution set of 2 conv layers followed by max pooling has 64 filters each
model.add(Conv2D(64, (3, 3), padding='same',activation='relu', name='set2_conv3'))
model.add(Conv2D(64, (3, 3), activation='relu', name='set2_conv4'))
model.add(MaxPooling2D(pool_size=(2, 2), name='set2_maxpool2'))
model.add(Dropout(0.25))

#Adding fully connected layer
model.add(Flatten(name='flatten'))
model.add(Dense(512, activation='relu', name='fully_connected_layer1'))
model.add(Dropout(0.25))
model.add(Dense(len(classes), activation='softmax', name='prediction_layer'))

# adding a optimizer
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd , metrics=['accuracy'])
#keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
set1_conv1 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
set1_conv2 (Conv2D)          (None, 28, 28, 32)        9248      
_________________________________________________________________
set1_maxpool1 (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
set2_conv3 (Conv2D)          (None, 14, 14, 64)        18496     
_________________________________________________________________
set2_conv4 (Conv2D)          (None, 12, 12, 64)        36928     
_________________________________________________________________
set2_maxpool2 (MaxPooling2D) (None, 6, 6, 64)          0         
__________

In [4]:
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.2, horizontal_flip=False, vertical_flip=False )
valid_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_path, target_size = (30,30), batch_size = batch_size, class_mode = 'categorical')
validation_set = valid_datagen.flow_from_directory(valid_path, target_size = (30,30), batch_size = batch_size, class_mode = 'categorical')

Found 2708 images belonging to 10 classes.
Found 250 images belonging to 10 classes.


In [5]:
model.fit_generator(training_set, steps_per_epoch=85 , epochs= epochs)
model.save_weights('/home/sree/Documents/task_cnn.h5')

Epoch 1/30
85/85 [==============================] - 37s 431ms/step - loss: 11.4470 - acc: 0.2895
Epoch 2/30
85/85 [==============================] - 29s 339ms/step - loss: 11.3715 - acc: 0.2945
Epoch 3/30
85/85 [==============================] - 45s 525ms/step - loss: 11.3715 - acc: 0.2945
Epoch 4/30
85/85 [==============================] - 45s 527ms/step - loss: 11.3750 - acc: 0.2943
Epoch 5/30
85/85 [==============================] - 44s 516ms/step - loss: 11.3715 - acc: 0.2945
Epoch 6/30
85/85 [==============================] - 45s 532ms/step - loss: 11.3715 - acc: 0.2945
Epoch 7/30
85/85 [==============================] - 44s 522ms/step - loss: 11.3750 - acc: 0.2943
Epoch 8/30
85/85 [==============================] - 49s 579ms/step - loss: 11.3750 - acc: 0.2943
Epoch 9/30
85/85 [==============================] - 46s 542ms/step - loss: 11.3575 - acc: 0.2954
Epoch 10/30
85/85 [==============================] - 46s 542ms/step - loss: 11.3715 - acc: 0.2945
Epoch 11/30
85/85 [==========

In [16]:
# choosing a random image to check our network accuracy of classification of test image data
random_file = random.choice(os.listdir(test_path))
file_path=test_path+'/'+random_file
Image.open(file_path).show()
#imagefile=file_path
#plt.imshow('imagefile')
test_image = image.load_img(file_path, target_size = (30, 30))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [17]:
label=training_set.class_indices
print(label)
# print the result of classifier, where test image (random) is fed to classifier to predict
print(result)
# result is given in the form of numpy array of class number
result.argmax(axis=1)
a = result[0] # Can be of any shape
indices = np.where(a == a.max())
print(indices)
result[0]
max(result[0])

{'Leopards': 0, 'Motorbikes': 1, 'airplanes': 2, 'bonsai': 3, 'car_side': 4, 'chandelier': 5, 'grand_piano': 6, 'hawksbill': 7, 'ketch': 8, 'watch': 9}
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
(array([1]),)


1.0